In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="3"

In [3]:
train_total = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [4]:
device = torch.device("cuda:0")
batch_size = 64
dropout_rate = 0.1
class_n = len(train_total['disease_code'].unique())
learning_rate = 1e-4
epochs = 50
save_path = 'models/base.pt'

In [5]:
# albumentations_transform = A.Compose([
#     A.OneOf([
#                           A.HorizontalFlip(p=1),
#                           A.RandomRotate90(p=1),
#                           A.VerticalFlip(p=1)            
#     ], p=1),
#     A.OneOf([
#                           A.MotionBlur(p=1),
#                           A.OpticalDistortion(p=1),
#                           A.GaussNoise(p=1)                 
#     ], p=1),
# ])

In [6]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train'):
        self.mode = mode
        self.files = files
        if mode == 'train':
            self.labels = labels
            
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        if self.mode == 'train':
            img = cv2.imread('data/train_imgs/'+self.files[i])
            img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)
            img = img.astype(np.float32)/255
            img = np.transpose(img, (2,0,1))
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'label' : torch.tensor(self.labels[i], dtype=torch.long)
            }
        else:
            img = cv2.imread('data/test_imgs/'+self.files[i])
            img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)
            img = img.astype(np.float32)/255
            img = np.transpose(img, (2,0,1))
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
            }

In [7]:
train = train_total.iloc[:200]
val = train_total.iloc[200:]

In [8]:
train_dataset = CustomDataset(train['img_path'].str.split('/').str[-1].values, train['disease_code'].values)
val_dataset = CustomDataset(val['img_path'].str.split('/').str[-1].values, val['disease_code'].values)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

test_dataset = CustomDataset(test['img_path'].str.split('/').str[-1], labels=None, mode='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

In [9]:
class CNN_Model(nn.Module):
    def __init__(self, class_n, rate=0.1):
        super(CNN_Model, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.dropout = nn.Dropout(rate)
        self.output_layer = nn.Linear(in_features=1000, out_features=class_n, bias=True)
    
    def forward(self, inputs):
        output = self.output_layer(self.dropout(self.model(inputs)))
        return output

In [10]:
model = CNN_Model(class_n).to(device)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [12]:
def train_step(batch_item, epoch, batch, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        return loss
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
            
        return loss

In [13]:
loss_plot, val_loss_plot = [], []

for epoch in range(epochs):
    total_loss, total_val_loss = 0, 0
    
    tqdm_dataset = tqdm(enumerate(train_dataloader))
    training = True
    for batch, batch_item in tqdm_dataset:
        batch_loss = train_step(batch_item, epoch, batch, training)
        total_loss += batch_loss
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(batch_loss.item()),
            'Total Loss' : '{:06f}'.format(total_loss/(batch+1))
        })
    loss_plot.append(total_loss/(batch+1))
    
    tqdm_dataset = tqdm(enumerate(val_dataloader))
    training = False
    for batch, batch_item in tqdm_dataset:
        batch_loss = train_step(batch_item, epoch, batch, training)
        total_val_loss += batch_loss
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Val Loss': '{:06f}'.format(batch_loss.item()),
            'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1))
        })
    val_loss_plot.append(total_val_loss/(batch+1))
    
    if np.min(val_loss_plot) == val_loss_plot[-1]:
        torch.save(model, save_path)

4it [00:15,  3.88s/it, Epoch=1, Loss=0.999866, Total Loss=1.308564]
1it [00:04,  4.08s/it, Epoch=1, Val Loss=0.944481, Total Val Loss=0.944481]
4it [00:15,  3.90s/it, Epoch=2, Loss=0.424079, Total Loss=0.157262]
1it [00:04,  4.06s/it, Epoch=2, Val Loss=0.688083, Total Val Loss=0.688083]
4it [00:15,  3.95s/it, Epoch=3, Loss=0.121874, Total Loss=0.054790]
1it [00:04,  4.07s/it, Epoch=3, Val Loss=0.500526, Total Val Loss=0.500526]
4it [00:15,  3.88s/it, Epoch=4, Loss=0.219723, Total Loss=0.067268]
1it [00:04,  4.10s/it, Epoch=4, Val Loss=0.459876, Total Val Loss=0.459876]
4it [00:15,  3.89s/it, Epoch=5, Loss=0.098606, Total Loss=0.036424]
1it [00:04,  4.16s/it, Epoch=5, Val Loss=0.514150, Total Val Loss=0.514150]
4it [00:15,  3.88s/it, Epoch=6, Loss=0.452386, Total Loss=0.148932]
1it [00:04,  4.04s/it, Epoch=6, Val Loss=0.535790, Total Val Loss=0.535790]
4it [00:15,  3.88s/it, Epoch=7, Loss=0.086329, Total Loss=0.025206]
1it [00:04,  4.10s/it, Epoch=7, Val Loss=0.659844, Total Val Loss=0.

In [15]:
def predict(dataset):
    model.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    training = False
    results = []
    for batch, batch_item in tqdm_dataset:
        img = batch_item['img'].to(device)
        with torch.no_grad():
            output = model(img)
        output = torch.tensor(torch.argmax(output, axis=-1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

In [16]:
preds = predict(test_dataloader)

submission = pd.read_csv('sample_submission.csv')
submission.iloc[:,1] = preds
submission.to_csv('baseline.csv', index=False)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
75it [05:31,  4.42s/it]
